# 0. Carregando os dados e bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from matplotlib import gridspec #Para fazer os dashs
#Foram adicionados por conta de erro de renderização no mapa (ex1.10), utilizando o VS Code. Caso sua IDE não tenha o erro, não é necessário
#Erro: No renderer could be found for mimetype "application/vnd.plotly.v1+json"
import plotly.io as pio
pio.renderers.default = "notebook_connected"
from geopy.geocoders import Nominatim
import ipywidgets as widgets
from ipywidgets import fixed

In [ ]:
data = pd.read_csv('datasets/kc_house_data.csv')

# 1. Respondendo às perguntas de negócio

## 1.1 Qual a quantidade de imóveis por nível?
 - Nivel 0: Preço entre R$ 0.00 e R$ 321.950
 - Nivel 1: Preço entre R$ 321.950 e R$ 450.000
 - Nivel 2: Preço entre R$ 450.000 e R$ 645.000
 - Nivel 3: Preço acima de R$ 645.000

In [ ]:
for i in range (len(data)):
    if (data.loc[i, 'price'] >= 0) & (data.loc[i, 'price'] <= 321950):
        data.loc[i, 'level'] = 'nivel_0'

    elif (data.loc[i, 'price'] > 321950) & (data.loc[i, 'price'] <= 450000):
        data.loc[i, 'level'] = 'nivel_1'

    elif (data.loc[i, 'price'] > 450000) & (data.loc[i, 'price'] <= 645000):
        data.loc[i, 'level'] = 'nivel_2'

    elif (data.loc[i, 'price'] > 645000) :
        data.loc[i, 'level'] = 'nivel_3'
        
    else:
        data[i, 'level'] = 'NA'

In [ ]:
data['level'].unique()

In [ ]:
data.groupby('level')['id'].count()

## 1.2 Adicione as seguintes informações ao imóvel:
 - O nome da Rua
 - O número do imóvel
 - O nome do Bairro
 - O nome da Cidade
 - O nome da Estado



##### Exemplo de uma chamada

In [ ]:
#Inicializar a API Nominatim
geolocator = Nominatim(user_agent='geoapiExercises')

response = geolocator.reverse('47.5112, -122.257') #nome padrão para guardar chamadas de API. Poderia ser outro nome.

In [ ]:
#Visualizar os dados da chamada de exemplo
response.raw

In [ ]:
data = pd.read_csv('datasets/kc_house_data.csv')

##### Respondendo à pergunta

In [ ]:
#Inicializar a API Nominatim
geolocator = Nominatim(user_agent='geoapiExercises')

for i in range(len (data)):
    print('Loop {} / {}'.format(i, len(data)))
    query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])

    #Chamada na API
    response = geolocator.reverse(query)

    #Popular as colunas
    #As condicionais foram feitas para evitar quebra de inserção, caso não haja alguma das informações pedidas na API
    if 'road' in response.raw['address']:
        data.loc[i, 'street_name']   = response.raw['address']['road']

    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number']  = response.raw['address']['house_number']

    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood'] = response.raw['address']['neighbourhood']

    if 'city' in response.raw['address']:
        data.loc[i, 'city']          = response.raw['address']['city']
        
    if 'state' in response.raw['address']:
        data.loc[i, 'state']         = response.raw['address']['state']

## 1.3 Adicionar o nível dos imóveis no Mapa como uma cor.



In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

houses = data[['id', 'lat', 'long', 'price']].copy()

#Definir níveis
for i in range(len(houses)):
    if houses.loc[i,'price'] <= 312950:
        houses.loc[i,'level'] = 0

    elif (houses.loc[i,'price'] > 321950) & (houses.loc[i,'price'] <= 450000):
        houses.loc[i, 'level'] = 1

    elif (houses.loc[i,'price'] > 450000) & (houses.loc[i,'price'] <= 645000):
        houses.loc[i, 'level'] = 2

    else:
        houses.loc[i,'level'] = 3

fig = px.scatter_mapbox(houses,
                        lat = 'lat',
                        lon = 'long',
                        color ='level',
                        size = 'price',
                        color_continuous_scale = px.colors.cyclical.IceFire,
                        size_max = 15,
                        zoom = 10

)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(height = 600, margin = {'r':0, 't':0, 'l':0, 'b':0})
fig.show()

## 1.4 Adicionar o Preço dos imóveis como o tamanho do ponto no mapa.



In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

houses = data[['id', 'lat', 'long', 'price']].copy()

#Definir níveis
for i in range(len(houses)):
    if houses.loc[i,'price'] <= 312950:
        houses.loc[i,'level'] = 0

    elif (houses.loc[i,'price'] > 321950) & (houses.loc[i,'price'] <= 450000):
        houses.loc[i, 'level'] = 1

    elif (houses.loc[i,'price'] > 450000) & (houses.loc[i,'price'] <= 645000):
        houses.loc[i, 'level'] = 2

    else:
        houses.loc[i,'level'] = 3

fig = px.scatter_mapbox(houses,
                        lat = 'lat',
                        lon = 'long',
                        color ='price',
                        color_continuous_scale = px.colors.cyclical.IceFire,
                        size_max = 15,
                        zoom = 10

)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(height = 600, margin = {'r':0, 't':0, 'l':0, 'b':0})
fig.show()

## 1.5 Adicionar opções de filtros para eu fazer minhas próprias análises:
- Eu quero escolher visualizar imóveis com vista para água ou não.
- Eu quero filtrar os imóveis até um certo valor de preço.



In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

data['is_waterfront'] = data['waterfront'].apply(lambda x: 'yes' if x == 1 else 'no')

data['level'] = data['price'].apply(lambda x:   0 if x <= 312950 else
                                                1 if (x > 321950) & (x <= 450000) else
                                                2 if (x > 450000) & (x <= 645000) else
                                                3 )  

style = {'description_width' : 'initial'}


price_limit = widgets.IntSlider( value = 540000,
                                 min = 75000,
                                 max = 77000000,
                                 step = 1, #variação quando o slice é mudado de posição
                                 description = 'Maior preço',
                                 disable = False,
                                 style = style
                                )

#Criando o botão de filtro de vista pra água
waterfront_bar = widgets.Dropdown(  options     = data['is_waterfront'].unique().tolist(),
                                    value       = 'yes',
                                    disable     = False,
                                    description = 'Vista água?'
                                  )


#Definindo a função que vai fazer com que os botões interajam com o mapa

def update_map (data, waterfront, limit):
    houses = data [ (data['price'] <= limit) & (data['is_waterfront'] == waterfront) ][['id', 'lat','long', 'price', 'level']] #botões para interagir com o mapa

    fig = px.scatter_mapbox(houses,
                            lat = 'lat',
                            lon = 'long',
                            color ='level',
                            size = 'price',
                            color_continuous_scale = px.colors.cyclical.IceFire,
                            size_max = 15,
                            zoom = 10

    )

    fig.update_layout(mapbox_style = 'open-street-map')
    fig.update_layout(height = 600, margin = {'r':0, 't':0, 'l':0, 'b':0})
    fig.show()

#Plot do mapa com os botões
widgets.interactive(update_map, data = fixed (data), waterfront = waterfront_bar, limit = price_limit)

## 1.6 Adicionar opções de filtros no último dashboard enviado:
- Eu quero visualizar somente valor a partir de um data disponível para compra.



In [ ]:
data = pd.read_csv('datasets\kc_house_data.csv')

#Alterar formato da coluna de data 
data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
data['date'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%m-%d' )
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )

#Criar o botão de disponibilidade a partir da data selecionada para o mapa
date_limit = widgets.SelectionSlider( options=data['date'].sort_values().unique().tolist(),
                                      value = '2014-12-01',
                                      description = 'Disponivel',
                                      continuous_update=False,
                                      orientation='horizontal',
                                      readout=True )

#Definir a função que cria o mapa com o botão
def update_map(data, limit):
    data = data[data['date'] >= limit].copy()

    fig = plt.figure(figsize = (12,12))
    specs = gridspec.GridSpec(ncols = 2, nrows = 2, figure = fig)

    ax1 = fig.add_subplot(specs[0,:])
    ax2 = fig.add_subplot(specs[1,0])
    ax3 = fig.add_subplot(specs[1,1])

    by_year = data[['id', 'year']].groupby( 'year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )

    by_day = data[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( 'title: Média de preço por dia' )

    by_week_of_year = data[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( 'title: Média de preço por semana do ano' )
    
    plt.xticks( rotation=60 )


widgets.interactive( update_map, data=fixed( data ), limit=date_limit)
